In [1]:
!python -V


Python 3.12.1


In [2]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("module-2-test-experiment")

2025/07/10 14:48:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/10 14:48:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/MLOPS-Zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1752156806878, experiment_id='2', last_update_time=1752156806878, lifecycle_stage='active', name='module-2-test-experiment', tags={}>

In [4]:
with mlflow.start_run():
    mlflow.log_param("learning_rate", 0.01)
    mlflow.log_metric("rmse", 0.87)


In [5]:
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)


(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 7.76


In [13]:
import pickle
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
import mlflow
from sklearn.linear_model import Lasso
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [20]:
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [jupyter]━━━ 7/9 [jupyter-console]


In [21]:
import xgboost as xgb

In [22]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/usr/local/python/3.12.1/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [26]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [27]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [28]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                                  | 0/50 [00:00<?, ?trial/s, best loss=?]

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:55:01] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.37029                                                                                   
[1]	validation-rmse:7.16250                                                                                   
[2]	validation-rmse:6.80492                                                                                   
[3]	validation-rmse:6.69033                                                                                   
[4]	validation-rmse:6.64734                                                                                   
[5]	validation-rmse:6.62671                                                                                   
[6]	validation-rmse:6.61285                                                                                   
[7]	validation-rmse:6.60695                                                                                   
[8]	validation-rmse:6.60294                                                                                   
[

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:55:58] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.78608                                                                                   
[3]	validation-rmse:7.35340                                                                                   
[4]	validation-rmse:7.11157                                                                                   
[5]	validation-rmse:6.97533                                                                                   
[6]	validation-rmse:6.89712                                                                                   
[7]	validation-rmse:6.85136                                                                                   
[8]	validation-rmse:6.82380                                                                                   
[9]	validation-rmse:6.80459                                                                                   
[10]	validation-rmse:6.79286                                                                                  
[

...

[0]	validation-rmse:10.95053                                                                                  
[1]	validation-rmse:9.94905                                                                                   
[2]	validation-rmse:9.16340                                                                                   
[3]	validation-rmse:8.55041                                                                                   
[4]	validation-rmse:8.07935                                                                                   
[5]	validation-rmse:7.72063                                                                                   
[6]	validation-rmse:7.44770                                                                                   
[7]	validation-rmse:7.23957                                                                                   
[8]	validation-rmse:7.08176                                                                                   
[

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [17:03:06] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:11.14378                                                                                  
[1]	validation-rmse:10.25919                                                                                  
[2]	validation-rmse:9.53273                                                                                   
[3]	validation-rmse:8.93795                                                                                   
[4]	validation-rmse:8.45675                                                                                   
[5]	validation-rmse:8.07153                                                                                   
[6]	validation-rmse:7.76042                                                                                   
[7]	validation-rmse:7.51291                                                                                   
[8]	validation-rmse:7.31587                                                                                   
[

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [17:05:49] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.78592                                                                                   
[1]	validation-rmse:7.45197                                                                                   
[2]	validation-rmse:6.96650                                                                                   
[3]	validation-rmse:6.77868                                                                                   
[4]	validation-rmse:6.69494                                                                                   
[5]	validation-rmse:6.65433                                                                                   
[6]	validation-rmse:6.63074                                                                                   
[7]	validation-rmse:6.61841                                                                                   
[8]	validation-rmse:6.60340                                                                                   
[